In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import csv
from selenium.common.exceptions import NoSuchElementException
import pandas as pd
import re

In [2]:
driver = webdriver.Chrome()  

In [3]:
#APP URL, add the url of the app
whatsapp = 'https://play.google.com/store/apps/details?id=com.whatsapp'
facebook = "https://play.google.com/store/apps/details?id=com.facebook.katana"
messenger = "https://play.google.com/store/apps/details?id=com.facebook.orca"
instagram = "https://play.google.com/store/apps/details?id=com.instagram.android"
subway_surfers = "https://play.google.com/store/apps/details?id=com.kiloo.subwaysurf"
candy_crush = "https://play.google.com/store/apps/details?id=com.king.candycrushsaga"
snap_chat = "https://play.google.com/store/apps/details?id=com.snapchat.android"
spotify = "https://play.google.com/store/apps/details?id=com.spotify.music"
my_talking_tom = "https://play.google.com/store/apps/details?id=com.outfit7.talkingtom"
twitter = "https://play.google.com/store/apps/details?id=com.twitter.android"

In [5]:
#add the app name and the url variable here
app_name = 'instagram'
driver.get(instagram)

After running the above cell, click the Rating and reviews arrow on the playstore app page on your browser, A modal containing reviews will apear, scroll down to as much reviews as you need, then generate.

In [6]:
#GENERATE REVIEWS
def scrap_app_reviews(app_name):
    #1. Get the Stars column
    #NOTE: Sign out of playstore before you start generating
    stars_column = []
    stars = driver.find_elements(By.CLASS_NAME, 'iXRFPc')
    i=0
    while True:
        try:
            star = stars[i+1].get_attribute("aria-label")
            stars_column.append(star[6])
            i+=1
        except IndexError:
            # we have reached the end of the elements
            break
    #first 2 stars are not part of the data so it should be removed.
    stars_column = stars_column[2:]
    
    #2. Get the Other columns (Name, Date, Review, Found_helpful)
    other_columns = []
    i = 0
    while True:
        xpath = "(//div[@class='RHo1pe'][{}])".format(i+1)
        try:
            elements = driver.find_elements(By.XPATH, xpath)[0]
            # do something with elements
            i += 1
            other_columns.append(elements.text)
        except IndexError:
            # we have reached the end of the elements
            break
    
    #3. Split using \n
    output = []
    for i in other_columns:
        row = i.split("\n")
        output.append(row)
    
    #4. Organize the data in a 2d list
    data = []
    for i in output:
        temp_output = []
        temp_output.append(i[0])
        temp_output.append(i[2])
        temp_output.append(i[3])
        if i[4] == 'Did you find this helpful?': #if 0 people found the review helpful, this will shift to the 4th index
            temp_output.append(0)
        else:
            temp_output.append(i[4])
        data.append(temp_output)
        
    #5. Convert to dataframe and combine the stars and others columns 
    final_data = pd.DataFrame(data, columns=['Name','Date', 'Review', 'Found Helpful'])
    final_data['Stars'] = stars_column

    #6. Create a solumn with the app name
    app_name_column = []
    for i in range(len(final_data)):
        i = app_name
        app_name_column.append(i)
    final_data['App Name'] = app_name_column
    
    #7. Save as csv
    final_data.to_csv(app_name, index=False)



In [7]:
scrap_app_reviews(app_name)

# Combine Reviews Into on file

In [126]:
candy_crush = pd.read_csv("candy_crush")
whatsapp = pd.read_csv("whatsapp")
facebook = pd.read_csv("facebook")
messenger = pd.read_csv("messenger")
instagram = pd.read_csv("instagram")
subway_surfers = pd.read_csv("subway_surfers")
snap_chat = pd.read_csv("snap_chat")
spotify = pd.read_csv("spotify")
my_talking_tom = pd.read_csv("my_talking_tom")
twitter = pd.read_csv("twitter")

In [145]:
app_reviews_dataset = pd.concat([candy_crush,whatsapp,facebook,messenger,instagram,subway_surfers,snap_chat,spotify,my_talking_tom,twitter])

In [148]:
app_reviews_dataset
app_reviews_dataset.drop('Unnamed: 0', axis=1, inplace=True)

In [173]:
app_reviews_dataset.to_csv('app_reviews_dataset', index=False)

In [172]:
app_reviews_dataset

,Name,Date,Review,Found Helpful,Stars,App Name
0,Ajinkya Nikam,2023-02-19,The algorithm makes sure you have to purchase ...,928,1,candy_crush
1,Anuj Bagde,2023-01-13,Ad not available... It's getting old watching ...,11556,5,candy_crush
2,Mohd Asif,2023-01-21,They have stopped ads for lives and extra move...,7837,1,candy_crush
3,Amandeep Singh,2022-12-13,I love this game very much. I love the colourf...,21273,5,candy_crush
4,Usha Sehgal,2022-12-27,I've finished level 13190.But for the last 4 w...,9703,1,candy_crush
...,...,...,...,...,...,...
2475,Dutch Oldies,2023-01-30,It's very valuable because it provides you wit...,0,4,twitter
2476,Waleed Saeid,2023-02-08,"Recently on the mobile version, The like icon ...",0,4,twitter
2477,legends only,2022-12-18,Wth Twitter! Why am I getting notification fro...,2,3,twitter
2478,Chris Thorn,2022-10-20,Having a little trouble with the app at this t...,52,3,twitter


# Clean

In [ ]:
app_reviews_dataset['Found Helpful'] = df['Found Helpful'].str.extract('(\d+)').astype(int)

In [154]:
app_reviews_dataset['Found Helpful'] = app_reviews_dataset['Found Helpful'].apply(lambda x: int(''.join(filter(str.isdigit, str(x)))) if isinstance(x, str) else np.nan)

In [165]:
if app_reviews_dataset['Found Helpful'].dtype == app_reviews_dataset['Found Helpful'].apply(type).unique()[0]:
    print("All values in 'col2' are of the same data type")
else:
    print("Values in 'col2' are of different data types")

All values in 'col2' are of the same data type


In [164]:
app_reviews_dataset['Found Helpful'] = app_reviews_dataset['Found Helpful'].astype(int)

In [168]:
app_reviews_dataset['Stars'] = app_reviews_dataset['Stars'].fillna(0)

In [169]:
app_reviews_dataset['Stars'] = app_reviews_dataset['Stars'].astype()

In [171]:
app_reviews_dataset['Date'] = pd.to_datetime(app_reviews_dataset['Date'])

In [ ]:
driver.quit()